<a href="https://colab.research.google.com/github/rifan-refun/ml-study-case-1/blob/main/Project_Group_2_GDSC_UIN_Jakarta_Mall_Customer_Visits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Study Jams GDSC UIN Jakarta: Multiclass Classification**

*Kelompok 2 Final Project*

## **Summary & Goals**

This project predicts which days of the week customers will likely revisit the mall. The goal is to classify and provide predictions for the specific day of the week, such as Monday, Tuesday, and so forth, when customers are expected to return.

## **Data Preparation**

These steps are the steps needed to prepare the data before it is used in the modelling process.

### **Import Libraries**

Install all required libraries using the import function.

In [ ]:
# additional library
!pip install -q gdown==4.6.0

In [ ]:
# import library
import os
import scipy
import gdown
import shutil
import random
import pickle
import zipfile
import scipy.stats
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import scipy.cluster.hierarchy as hc
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix

# warnings
import warnings
warnings.simplefilter('ignore', category=FutureWarning)
warnings.simplefilter('ignore', category=UserWarning)

In [ ]:
# function to display all in dataframe
def display_all(dataframe):
    with pd.option_context('display.max_rows', 1000, 'display.max_columns', 1000):
        display(dataframe)

### **Data Downloading**

In [ ]:
# url datasets
url_drive = 'https://drive.google.com/drive/folders/1DqG1WdMU-khh5rtLtd07NS-QfWLAQQ35'
url_repository = 'https://github.com/nurmuhimawann/mall-customer-visits/releases/download/v.1.0.0/datasets-v1.0.0.zip'

# download datasets
try:
    gdown.download_folder(url_drive, quiet=False)
except Exception as e:
    print("Error downloading from Google Drive:", e)
    print("\nAttempting to download from the repository...")

    # try downloading from repository
    try:
        !wget --no-check-certificate $url_repository -O mall-customer-visits.zip
        print("Download completed")
    except Exception as e:
        print("Error downloading from the repository:", e)
        print("No dataset available.")

Retrieving folder list


Processing file 1LVjkTlC9wnPWx6GVZU3vsJlcKFkrdU02 test_set.csv
Processing file 108h4SCNBdm0R0R2R4AtWLBUD-vflQCrj train_set.csv
Building directory structure completed


Retrieving folder list completed
Building directory structure
Downloading...
From: https://drive.google.com/uc?id=1LVjkTlC9wnPWx6GVZU3vsJlcKFkrdU02
To: /content/Mall - Customer Visit/test_set.csv
100%|██████████| 613/613 [00:00<00:00, 610kB/s]
Downloading...
From: https://drive.google.com/uc?id=108h4SCNBdm0R0R2R4AtWLBUD-vflQCrj
To: /content/Mall - Customer Visit/train_set.csv
100%|██████████| 13.0k/13.0k [00:00<00:00, 23.4MB/s]
Download completed


### **Directory Specification**

Specify the directories that are going to be used where the dataset is stored.

In [ ]:
# define directory checkpoints
checkpoint_dir = '/content/checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# directory specification
base_dir = r"/content/Mall - Customer Visit"
os.listdir(base_dir)

['test_set.csv', 'train_set.csv']

## **Data Wrangling**

### **Data Loading**

In [ ]:
# read csv
data = pd.read_csv(os.path.join(base_dir, 'train_set.csv'))
data.head()

,visitor_id,visits
0,1,30 84 126 135 137 179 242 342 426 456 460 462 ...
1,2,24 53 75 134 158 192 194 211 213 238 251 305 4...
2,3,51 143 173 257 446 491 504 510 559 616 719 735...
3,4,1 20 22 92 124 149 211 335 387 390 406 460 489...
4,5,34 51 56 106 110 121 163 233 266 275 345 359 3...


### **Assesing Data**

#### **Data Information**

In [ ]:
# info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   visitor_id  99 non-null     int64 
 1   visits      99 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


#### **Checking Missing Values**

In [ ]:
# check missing values
data.isnull().sum()

visitor_id    0
visits        0
dtype: int64

#### **Checking Duplicate Values**

In [ ]:
# check duplicate
data.duplicated().sum()

0

#### **Data Cleaning**

Based on the provided data information, it appears that the visit data is stored as a list object. Therefore, our first step would be to normalize the data by storing each visit in a separate row.

In [ ]:
# remove leading and trailing whitespace from each data in the 'visits' column
# split each string into a list based on whitespace and expand it into separate columns in dataframe
visits_df = data['visits'].str.strip().str.split(expand=True)
visits_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,30,84,126,135,137,179,242,342,426,456,...,None,None,None,None,None,None,None,None,None,None
1,24,53,75,134,158,192,194,211,213,238,...,None,None,None,None,None,None,None,None,None,None
2,51,143,173,257,446,491,504,510,559,616,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
# merge the main data with the expanded visits
df_merged = (data.merge(visits_df, right_index=True, left_index=True)
                .drop(['visits'], axis=1)
                .melt(id_vars=['visitor_id'], value_name="visit_day")
                .drop("variable", axis=1)
                .dropna()
          )

# convert 'visit_day' dtype from object to int
df_merged['visit_day'] = df_merged['visit_day'].astype('int')

# sort and reset idx by 'visitor_id'
df_raw = (df_merged.sort_values(['visitor_id', 'visit_day']).reset_index(drop=True))

df_raw

,visitor_id,visit_day
0,1,30
1,1,84
2,1,126
3,1,135
4,1,137
...,...,...
3224,99,907
3225,99,911
3226,99,914
3227,99,919


Saving the raw data into a feather file is a checkpoint, enabling it to quickly revert to this state if needed or if anything goes wrong.

In [ ]:
# save as data checkpoint
df_raw.to_feather('/content/checkpoints/df_raw')

# checking files
if os.path.exists('/content/checkpoints/df_raw'):
    print(f"The file 'df_raw' is saved.")

The file 'df_raw' is saved.


Delete all the unused object from memory. We will need to be memory efficient throughout

In [ ]:
# delete unusued vars
del df_merged
del df_raw
del visits_df

Now that the data is in a more analytics-friendly format, we can proceed with Exploratory Data Analysis (EDA) and feature engineering.

## **Exploratory Data Analytics (EDA) & Feature Engineering**

In [ ]:
# load dataframe from the checkpoint directory
df_visits = pd.read_feather('/content/checkpoints/df_raw')
df_visits.head()

,visitor_id,visit_day
0,1,30
1,1,84
2,1,126
3,1,135
4,1,137


### **Distribution of Visits**

In [ ]:
# Distribution of visit days
fig = go.Figure(data=[go.Histogram(x=df_visits['visit_day'], nbinsx=1001)])

# layout settings
layout_settings = dict(
    title='Distribution of Visits',
    title_font_size=18,
    template='plotly_white',
    xaxis=dict(title='Visit Day'),
    yaxis=dict(title='Frequency')
)
fig.update_layout(layout_settings)
fig.update_traces(hovertemplate='visit_day=%{x}<br>count=%{y}')
fig.show()

We can observe seasonal patterns in the data, where it is likely that many visits occur on weekends. Let's explore this further, including information on the day of the week.

### **Distribution of visits by day of week**

In [ ]:
# mapping to day of week
df_visits['day_of_week'] = df_visits['visit_day'].apply(lambda values: (values % 7) or 7)
df_visits.head()

,visitor_id,visit_day,day_of_week
0,1,30,2
1,1,84,7
2,1,126,7
3,1,135,2
4,1,137,4


In [ ]:
# Distribution of visits by day of week
day_mapping = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7: 'Sunday'}
top_days = df_visits['day_of_week'].value_counts().nlargest(2).index
colors = ['#636EFA' if day in top_days else 'lightslategray' for day in df_visits['day_of_week'].value_counts().index]

fig = go.Figure(data=[go.Bar(
    x=[day_mapping[day] for day in df_visits['day_of_week'].value_counts().index],
    y=df_visits['day_of_week'].value_counts().values,
    marker_color=colors,
    text=df_visits['day_of_week'].value_counts().values,
    textposition='outside'
)])

# layout settings
layout_settings = dict(
    title='Distribution of Visits by Day of Week',
    title_font_size=18,
    template='plotly_white',
    xaxis=dict(title='Day of Week', categoryorder='array', categoryarray=[day_mapping[day] for day in range(1, 8)]),
    yaxis=dict(title='Frequency')
)
fig.update_layout(layout_settings)
fig.update_traces(hovertemplate='visit_day=%{x}<br>count=%{y}')
fig.show()

Our hypothesis seems correct, and weekends are the most popular days for mall visits.

In [ ]:
df_visits['month_position'] = (df_visits['visit_day'] // 30)
df_visits

,visitor_id,visit_day,day_of_week,month_position
0,1,30,2,1
1,1,84,7,2
2,1,126,7,4
3,1,135,2,4
4,1,137,4,4
...,...,...,...,...
3224,99,907,4,30
3225,99,911,1,30
3226,99,914,4,30
3227,99,919,2,30


In [ ]:
# Count visits for each month
monthly_counts = df_visits['month_position'].value_counts().sort_index()

# Create Plotly figure
fig = go.Figure()

# Add bar trace
fig.add_trace(go.Bar(
    x=monthly_counts.index,
    y=monthly_counts.values,
    marker_color='skyblue'
))

# Update layout
fig.update_layout(
    xaxis_title='Month',
    yaxis_title='Visit Count',
    title='Visits Count for Each Month',
    xaxis=dict(tickmode='array', tickvals=list(monthly_counts.index)),
    yaxis=dict(gridcolor='lightgrey'),
    plot_bgcolor='rgba(0,0,0,0)'
)

# Show plot
fig.show()

### **Create week_number**

Week number represents the sequence of weeks in which visitors visit the mall.

In [ ]:
# add week number
df_visits['week_number'] = (df_visits['visit_day'] - 1) // 7 + 1
df_visits.head()

,visitor_id,visit_day,day_of_week,month_position,week_number
0,1,30,2,1,5
1,1,84,7,2,12
2,1,126,7,4,18
3,1,135,2,4,20
4,1,137,4,4,20


In [ ]:
# Count visits for each month
monthly_counts = df_visits['week_number'].value_counts().sort_index()

# Create Plotly figure
fig = go.Figure()

# Add bar trace
fig.add_trace(go.Bar(
    x=monthly_counts.index,
    y=monthly_counts.values,
    marker_color='skyblue'
))

# Update layout
fig.update_layout(
    xaxis_title='Weekly',
    yaxis_title='Visit Count',
    title='Visits Count for Each Week',
    xaxis=dict(tickmode='array', tickvals=list(monthly_counts.index)),
    yaxis=dict(gridcolor='lightgrey'),
    plot_bgcolor='rgba(0,0,0,0)'
)

# Show plot
fig.show()

### **One-Hot Encoding for each day of week**

In [ ]:
# ohe for each day of week
df_visits = pd.concat([df_visits, pd.get_dummies(df_visits['day_of_week'], prefix='day')], axis=1)
df_visits.head()

,visitor_id,visit_day,day_of_week,month_position,week_number,day_1,day_2,day_3,day_4,day_5,day_6,day_7
0,1,30,2,1,5,0,1,0,0,0,0,0
1,1,84,7,2,12,0,0,0,0,0,0,1
2,1,126,7,4,18,0,0,0,0,0,0,1
3,1,135,2,4,20,0,1,0,0,0,0,0
4,1,137,4,4,20,0,0,0,1,0,0,0


### **Aggregate to weekly**

Aggregate this data into weekly intervals. Our goal is to predict the visit for next week.

In [ ]:
# aggregate to weekly
df_weekly = df_visits.groupby(['visitor_id', 'week_number']).agg({f'day_{i}': 'sum' for i in range(1, 8)})
df_weekly.head()

day_1  day_2  day_3  day_4  day_5  day_6  day_7
visitor_id week_number                                                 
1          5                0      1      0      0      0      0      0
           12               0      0      0      0      0      0      1
           18               0      0      0      0      0      0      1
           20               0      1      0      1      0      0      0
           26               0      0      0      1      0      0      0

In [ ]:
# descriptive statistic
df_weekly = df_weekly.reset_index()
df_weekly.describe()

,visitor_id,week_number,day_1,day_2,day_3,day_4,day_5,day_6,day_7
count,2912.000000,2912.000000,2912.000000,2912.000000,2912.000000,2912.000000,2912.000000,2912.000000,2912.000000
mean,51.902816,72.815247,0.153159,0.167926,0.146635,0.139080,0.158997,0.172390,0.170673
std,28.107718,41.196643,0.360203,0.373865,0.353802,0.346089,0.365736,0.377784,0.376288
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,28.000000,37.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,52.000000,73.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,76.000000,109.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,99.000000,143.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


From here, we know that there were 143 weeks and 99 visitors. We will use this information to process further data.

### **Creates the full range of weeks**

We need to add additional weeks to the data to represent those weeks where no visit happened.

In [ ]:
# creating the full time range dataframe
visitor_idx = pd.Series(range(1, 100)) # 99 visitors
visitor_idx_repeat = visitor_idx.repeat(143).reset_index(drop=True) # 143 weeks
temp_df = pd.DataFrame({'visitor_id': visitor_idx_repeat})
temp_df['record'] = 1
temp_df['week_number'] = temp_df.groupby('visitor_id')['record'].cumsum()

# Merging the full range with existing weekly data
df_weekly = df_weekly.merge(temp_df[['visitor_id', 'week_number']],
                            on=['visitor_id', 'week_number'],
                            how='outer').fillna(0).sort_values(['visitor_id', 'week_number'])

df_weekly

,visitor_id,week_number,day_1,day_2,day_3,day_4,day_5,day_6,day_7
2912,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2913,1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2914,1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2915,1,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,1,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
14152,99,139,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14153,99,140,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14154,99,141,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14155,99,142,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# info
df_weekly.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14157 entries, 2912 to 14156
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   visitor_id   14157 non-null  int64  
 1   week_number  14157 non-null  int64  
 2   day_1        14157 non-null  float64
 3   day_2        14157 non-null  float64
 4   day_3        14157 non-null  float64
 5   day_4        14157 non-null  float64
 6   day_5        14157 non-null  float64
 7   day_6        14157 non-null  float64
 8   day_7        14157 non-null  float64
dtypes: float64(7), int64(2)
memory usage: 1.1 MB


### **Count total_visits_in_week**

Add a column named 'total_visits_in_week' to denote the aggregate count of visits within each specific week.

In [ ]:
# add column for total visits in the week
df_weekly['total_visits_in_week'] = df_weekly.filter(like='day').sum(axis=1)

# downcast float to integer & reset idx
float_cols = df_weekly.select_dtypes(include=['float']).columns
df_weekly[float_cols] = df_weekly[float_cols].astype(np.uint32)
df_weekly.reset_index(drop=True, inplace=True)
df_weekly.head()

,visitor_id,week_number,day_1,day_2,day_3,day_4,day_5,day_6,day_7,total_visits_in_week
0,1,1,0,0,0,0,0,0,0,0
1,1,2,0,0,0,0,0,0,0,0
2,1,3,0,0,0,0,0,0,0,0
3,1,4,0,0,0,0,0,0,0,0
4,1,5,0,1,0,0,0,0,0,1


### **Count frequency of visits**

Generate the visit frequency up to a specific point in time by accumulating the total visits per week for each visitor

In [ ]:
# calculate frequency of visits for each visitor
df_weekly['freq'] = df_weekly.groupby('visitor_id')['total_visits_in_week'].cumsum().astype(np.uint32)

# keeps the rows where visitors have made at least one visit
df_weekly = df_weekly[df_weekly['freq'] != 0].reset_index(drop=True)
df_weekly.head()

,visitor_id,week_number,day_1,day_2,day_3,day_4,day_5,day_6,day_7,total_visits_in_week,freq
0,1,5,0,1,0,0,0,0,0,1,1
1,1,6,0,0,0,0,0,0,0,0,1
2,1,7,0,0,0,0,0,0,0,0,1
3,1,8,0,0,0,0,0,0,0,0,1
4,1,9,0,0,0,0,0,0,0,0,1


### **Any visit status**

We need to add a new column to specify whether a visit occurred on any day within the current week.

In [ ]:
# Add a column indicating if any visit occurred in the week
df_weekly['any_visit_status'] = (df_weekly['total_visits_in_week'] > 0).astype(np.uint32)
df_weekly.head()

,visitor_id,week_number,day_1,day_2,day_3,day_4,day_5,day_6,day_7,total_visits_in_week,freq,any_visit_status
0,1,5,0,1,0,0,0,0,0,1,1,1
1,1,6,0,0,0,0,0,0,0,0,1,0
2,1,7,0,0,0,0,0,0,0,0,1,0
3,1,8,0,0,0,0,0,0,0,0,1,0
4,1,9,0,0,0,0,0,0,0,0,1,0


### **no_visit**

Corresponding to no visits can be identified by subtracting the 'any_visit_status' column from 1.

In [ ]:
# Create a column indicating if no visit occurred
df_weekly['no_visits'] = 1 - df_weekly['any_visit_status']
df_weekly.head()

,visitor_id,week_number,day_1,day_2,day_3,day_4,day_5,day_6,day_7,total_visits_in_week,freq,any_visit_status,no_visits
0,1,5,0,1,0,0,0,0,0,1,1,1,0
1,1,6,0,0,0,0,0,0,0,0,1,0,1
2,1,7,0,0,0,0,0,0,0,0,1,0,1
3,1,8,0,0,0,0,0,0,0,0,1,0,1
4,1,9,0,0,0,0,0,0,0,0,1,0,1


### **weeks_since_previous_visit**

Your previous visit may affect your upcoming visit. Therefore let's get weeks since the last visit.

In [ ]:
# function to calculate weeks since prev visit
def calculate_weeks_since_prev_visit(visit_statuses):
    num_weeks_since_prev_visit = []
    num_visit = 0

    for status in visit_statuses:
        if status == 1:
            num_visit = 1
        else:
            num_visit += 1
        num_weeks_since_prev_visit.append(num_visit)

    return num_weeks_since_prev_visit

# apply function
num_weeks_since_prev_visit = calculate_weeks_since_prev_visit(df_weekly['any_visit_status'])

# concate to main data
weeks_since_prev_visit = pd.DataFrame({'weeks_since_prev_visit': num_weeks_since_prev_visit})
df_weekly = pd.concat([df_weekly, weeks_since_prev_visit], axis=1)
df_weekly['weeks_since_prev_visit'] = df_weekly['weeks_since_prev_visit'].astype(np.uint32)
df_weekly.head()

,visitor_id,week_number,day_1,day_2,day_3,day_4,day_5,day_6,day_7,total_visits_in_week,freq,any_visit_status,no_visits,weeks_since_prev_visit
0,1,5,0,1,0,0,0,0,0,1,1,1,0,1
1,1,6,0,0,0,0,0,0,0,0,1,0,1,2
2,1,7,0,0,0,0,0,0,0,0,1,0,1,3
3,1,8,0,0,0,0,0,0,0,0,1,0,1,4
4,1,9,0,0,0,0,0,0,0,0,1,0,1,5


### **Total visit for specific day of week from each visitor**

Get the total number of visits for a specific day of the week. The logic behind this feature is that someone visiting on Sundays is more likely to return.

In [ ]:
# define the relevant columns
columns_to_sum = ['no_visits', 'day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'day_7']

# calculate total visit for specific day of week
for column in columns_to_sum:
    new_column_name = f'tot_{column}'
    df_weekly[new_column_name] = df_weekly.groupby('visitor_id')[column].cumsum().astype(np.uint32)

display_all(df_weekly.head())

,visitor_id,week_number,day_1,day_2,day_3,day_4,day_5,day_6,day_7,total_visits_in_week,freq,any_visit_status,no_visits,weeks_since_prev_visit,tot_no_visits,tot_day_1,tot_day_2,tot_day_3,tot_day_4,tot_day_5,tot_day_6,tot_day_7
0,1,5,0,1,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0,0,0,0
1,1,6,0,0,0,0,0,0,0,0,1,0,1,2,1,0,1,0,0,0,0,0
2,1,7,0,0,0,0,0,0,0,0,1,0,1,3,2,0,1,0,0,0,0,0
3,1,8,0,0,0,0,0,0,0,0,1,0,1,4,3,0,1,0,0,0,0,0
4,1,9,0,0,0,0,0,0,0,0,1,0,1,5,4,0,1,0,0,0,0,0


### **Day of week proportion**

Get the proportion of days of the week of total visits and divide it by the frequency. Non-visits are calculated slightly differently because we want the proportion of non-visits to fall outside the whole time range.

In [ ]:
# define the relevant columns
day_columns = ['day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'day_7']

# calculate proportion of visits for each day
for day_column in day_columns:
    prop_column_name = f'prop_visit_{day_column}'
    df_weekly[prop_column_name] = df_weekly[f'tot_{day_column}'] / df_weekly['freq']

# calculate proportion of no visits
df_weekly['prop_no_visits'] = df_weekly['tot_no_visits'] / (df_weekly['freq'] + df_weekly['tot_no_visits'])

# downcast dtype to float
numeric_columns = ['prop_no_visits'] + [f'prop_visit_{day_column}' for day_column in day_columns]
df_weekly[numeric_columns] = df_weekly[numeric_columns].apply(pd.to_numeric, downcast='float')

display_all(df_weekly.head())

,visitor_id,week_number,day_1,day_2,day_3,day_4,day_5,day_6,day_7,total_visits_in_week,freq,any_visit_status,no_visits,weeks_since_prev_visit,tot_no_visits,tot_day_1,tot_day_2,tot_day_3,tot_day_4,tot_day_5,tot_day_6,tot_day_7,prop_visit_day_1,prop_visit_day_2,prop_visit_day_3,prop_visit_day_4,prop_visit_day_5,prop_visit_day_6,prop_visit_day_7,prop_no_visits
0,1,5,0,1,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,1,6,0,0,0,0,0,0,0,0,1,0,1,2,1,0,1,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.500000
2,1,7,0,0,0,0,0,0,0,0,1,0,1,3,2,0,1,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.666667
3,1,8,0,0,0,0,0,0,0,0,1,0,1,4,3,0,1,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.750000
4,1,9,0,0,0,0,0,0,0,0,1,0,1,5,4,0,1,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.800000


### **Create data labels**

In [ ]:
# create labels using nested ternary expressions
df_weekly['labels'] = np.where(df_weekly['day_1'] == 1, "Monday",
                               np.where(df_weekly['day_2'] == 1, "Tuesday",
                                        np.where(df_weekly['day_3'] == 1, "Wednesday",
                                                 np.where(df_weekly['day_4'] == 1, "Thursday",
                                                          np.where(df_weekly['day_5'] == 1, "Friday",
                                                                   np.where(df_weekly['day_6'] == 1, "Saturday",
                                                                            np.where(df_weekly['day_7'] == 1, "Sunday",
                                                                                     np.where(df_weekly['no_visits'] == 1, "No Visit", "")
                                                                            )
                                                                   )
                                                          )
                                                 )
                                        )
                                )
                       )

display_all(df_weekly.head())

,visitor_id,week_number,day_1,day_2,day_3,day_4,day_5,day_6,day_7,total_visits_in_week,freq,any_visit_status,no_visits,weeks_since_prev_visit,tot_no_visits,tot_day_1,tot_day_2,tot_day_3,tot_day_4,tot_day_5,tot_day_6,tot_day_7,prop_visit_day_1,prop_visit_day_2,prop_visit_day_3,prop_visit_day_4,prop_visit_day_5,prop_visit_day_6,prop_visit_day_7,prop_no_visits,labels
0,1,5,0,1,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,Tuesday
1,1,6,0,0,0,0,0,0,0,0,1,0,1,2,1,0,1,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.500000,No Visit
2,1,7,0,0,0,0,0,0,0,0,1,0,1,3,2,0,1,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.666667,No Visit
3,1,8,0,0,0,0,0,0,0,0,1,0,1,4,3,0,1,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.750000,No Visit
4,1,9,0,0,0,0,0,0,0,0,1,0,1,5,4,0,1,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.800000,No Visit


In [ ]:
# export
df_weekly.to_feather('/content/checkpoints/df_weekly')

# checking files
if os.path.exists('/content/checkpoints/df_weekly'):
    print(f"The file 'df_weekly' is saved.")

The file 'df_weekly' is saved.


In [ ]:
# remove unusued vars
del data
del df_visits
del df_weekly
del weeks_since_prev_visit

### **Cleaned up some features and optimizations**

In [ ]:
# read feather data
df_weekly = pd.read_feather('/content/checkpoints/df_weekly')

In [ ]:
# get features needed
features_tot_dow = []
features_prop_dow = []

for column in df_weekly.columns:
    if column.startswith('tot_'):
        features_tot_dow.append(column)
    elif column.startswith('prop_'):
        features_prop_dow.append(column)

features_other = ['freq', 'weeks_since_prev_visit']
features = features_tot_dow + features_prop_dow + features_other
features

['tot_no_visits',
 'tot_day_1',
 'tot_day_2',
 'tot_day_3',
 'tot_day_4',
 'tot_day_5',
 'tot_day_6',
 'tot_day_7',
 'prop_visit_day_1',
 'prop_visit_day_2',
 'prop_visit_day_3',
 'prop_visit_day_4',
 'prop_visit_day_5',
 'prop_visit_day_6',
 'prop_visit_day_7',
 'prop_no_visits',
 'freq',
 'weeks_since_prev_visit']

In [ ]:
# shift features 1 period
df_weekly[features] = df_weekly.groupby('visitor_id')[features].shift(1)

# downcast to float dtype
df_weekly_float = df_weekly.select_dtypes(include=['float']).columns
df_weekly[df_weekly_float] = df_weekly[df_weekly_float].apply(pd.to_numeric, downcast='float')

# remove rows with NaN frequency and reset idx
df_weekly = df_weekly.loc[~(df_weekly['freq'].isnull())].reset_index(drop = True)

display_all(df_weekly.head())

,visitor_id,week_number,day_1,day_2,day_3,day_4,day_5,day_6,day_7,total_visits_in_week,freq,any_visit_status,no_visits,weeks_since_prev_visit,tot_no_visits,tot_day_1,tot_day_2,tot_day_3,tot_day_4,tot_day_5,tot_day_6,tot_day_7,prop_visit_day_1,prop_visit_day_2,prop_visit_day_3,prop_visit_day_4,prop_visit_day_5,prop_visit_day_6,prop_visit_day_7,prop_no_visits,labels
0,1,6,0,0,0,0,0,0,0,0,1.0,0,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,No Visit
1,1,7,0,0,0,0,0,0,0,0,1.0,0,1,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.500000,No Visit
2,1,8,0,0,0,0,0,0,0,0,1.0,0,1,3.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.666667,No Visit
3,1,9,0,0,0,0,0,0,0,0,1.0,0,1,4.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.750000,No Visit
4,1,10,0,0,0,0,0,0,0,0,1.0,0,1,5.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.800000,No Visit


In [ ]:
# info
df_weekly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13631 entries, 0 to 13630
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   visitor_id              13631 non-null  int64  
 1   week_number             13631 non-null  int64  
 2   day_1                   13631 non-null  uint32 
 3   day_2                   13631 non-null  uint32 
 4   day_3                   13631 non-null  uint32 
 5   day_4                   13631 non-null  uint32 
 6   day_5                   13631 non-null  uint32 
 7   day_6                   13631 non-null  uint32 
 8   day_7                   13631 non-null  uint32 
 9   total_visits_in_week    13631 non-null  uint32 
 10  freq                    13631 non-null  float32
 11  any_visit_status        13631 non-null  uint32 
 12  no_visits               13631 non-null  uint32 
 13  weeks_since_prev_visit  13631 non-null  float32
 14  tot_no_visits           13631 non-null

In [ ]:
# save data as checkpoint
df_weekly.to_feather('/content/checkpoints/df')

# checking files
if os.path.exists('/content/checkpoints/df'):
    print(f"The file 'df' is saved.")

The file 'df' is saved.


## **Data Preprocessing**

### **Splitting Data**

In [ ]:
# load dataframe from the checkpoint directory
df = pd.read_feather('/content/checkpoints/df')

In [ ]:
# splitting data
train = df.loc[df['week_number'].isin(range(130, 142))]
val = df.loc[df['week_number'] == 142]

testing_set = df.loc[df['week_number'] == 143]

# shape
print(train.shape)
print(val.shape)

print(testing_set.shape)

(1188, 31)
(99, 31)
(99, 31)


In [ ]:
# get features needed
features_tot_dow = []
features_prop_dow = []

for column in df_weekly.columns:
    if column.startswith('tot_'):
        features_tot_dow.append(column)
    elif column.startswith('prop_'):
        features_prop_dow.append(column)

features_other = ['freq', 'weeks_since_prev_visit']
features = features_tot_dow + features_prop_dow + features_other
features

['tot_no_visits',
 'tot_day_1',
 'tot_day_2',
 'tot_day_3',
 'tot_day_4',
 'tot_day_5',
 'tot_day_6',
 'tot_day_7',
 'prop_visit_day_1',
 'prop_visit_day_2',
 'prop_visit_day_3',
 'prop_visit_day_4',
 'prop_visit_day_5',
 'prop_visit_day_6',
 'prop_visit_day_7',
 'prop_no_visits',
 'freq',
 'weeks_since_prev_visit']

I will remove the features below as they are similar to other features and do not significantly impact the model.

In [ ]:
# remove some features
features.remove('tot_no_visits') # correlated with prop_no_visits
features.remove('freq') # correlated with prop_no_visits

features

['tot_day_1',
 'tot_day_2',
 'tot_day_3',
 'tot_day_4',
 'tot_day_5',
 'tot_day_6',
 'tot_day_7',
 'prop_visit_day_1',
 'prop_visit_day_2',
 'prop_visit_day_3',
 'prop_visit_day_4',
 'prop_visit_day_5',
 'prop_visit_day_6',
 'prop_visit_day_7',
 'prop_no_visits',
 'weeks_since_prev_visit']

In [ ]:
# define target variables
targets = ['labels']

In [ ]:
# define xy variables
X_train = train[features]
y_train = train[targets]

X_val = val[features]
y_val = val[targets]

X_test = testing_set[features]
y_test = testing_set[targets]

In [ ]:
# x_train
display_all(X_train.head())

,tot_day_1,tot_day_2,tot_day_3,tot_day_4,tot_day_5,tot_day_6,tot_day_7,prop_visit_day_1,prop_visit_day_2,prop_visit_day_3,prop_visit_day_4,prop_visit_day_5,prop_visit_day_6,prop_visit_day_7,prop_no_visits,weeks_since_prev_visit
124,1.0,5.0,1.0,5.0,2.0,4.0,4.0,0.045455,0.227273,0.045455,0.227273,0.090909,0.181818,0.181818,0.828125,3.0
125,1.0,5.0,1.0,5.0,2.0,4.0,4.0,0.045455,0.227273,0.045455,0.227273,0.090909,0.181818,0.181818,0.829457,4.0
126,1.0,5.0,1.0,5.0,2.0,4.0,4.0,0.045455,0.227273,0.045455,0.227273,0.090909,0.181818,0.181818,0.830769,5.0
127,1.0,5.0,1.0,5.0,2.0,4.0,5.0,0.043478,0.217391,0.043478,0.217391,0.086957,0.173913,0.217391,0.824427,1.0
128,1.0,5.0,1.0,6.0,2.0,4.0,5.0,0.041667,0.208333,0.041667,0.250000,0.083333,0.166667,0.208333,0.818182,1.0


In [ ]:
# y_train
display_all(y_train.head())

,labels
124,No Visit
125,No Visit
126,Sunday
127,Thursday
128,No Visit


### **Stardardize feature set**

In [ ]:
# standardize
scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

### **Check feature set correlations**

In [ ]:
# Compute correlation matrix
corr_matrix = X_train.corr(method='spearman')

# Calculate linkage matrix
corr_condensed = hc.distance.squareform(1 - corr_matrix)
z = hc.linkage(corr_condensed, method='average')

# dendrogram
fig = ff.create_dendrogram(X=corr_matrix.values, orientation='left', labels=corr_matrix.columns)
fig.update_layout(width=1400, height=700, title='Hierarchical of Correlation Matrix')
fig.show()

In [ ]:
# Compute correlation matrix
correlation = X_train.corr()

# heatmap
fig = go.Figure(data=go.Heatmap(
    z=correlation.values,
    x=correlation.columns,
    y=correlation.columns,
    colorscale='blues',
    colorbar=dict(title='Correlation')
))

# Update layout
fig.update_layout(
    title='Correlation Heatmap',
    xaxis=dict(title='Features'),
    yaxis=dict(title='Features'),
    margin=dict(l=200),
    height=750

)

fig.show()

## **Modelling**

### **Logistic Regression**

In [ ]:
# logistic regression
logistic_model = LogisticRegression(multi_class='ovr')

# Train the model
logistic_model.fit(X_train, y_train)

LogisticRegression(multi_class='ovr')

In [ ]:
# Predict on the validation set
y_train_pred = logistic_model.predict(X_train)

# Predict on the validation set
y_val_pred = logistic_model.predict(X_val)

# Predict on the testing set
y_test_pred = logistic_model.predict(X_test)

## **Model Evaluation**

In [ ]:
# Convert column to list of strings
y_train = y_train['labels'].tolist()
y_val = y_val['labels'].tolist()
y_test = y_test['labels'].tolist()

# Evaluation on training set
print("Training Set:")
print(classification_report(y_train, y_train_pred))
print("Accuracy:", accuracy_score(y_train, y_train_pred))

# Compute ROC AUC score for each class on training set
roc_auc_scores_train = roc_auc_score(y_train, logistic_model.predict_proba(X_train), average='macro', multi_class='ovr')
print("ROC AUC Scores (Training):", roc_auc_scores_train)

# Evaluation on validation set
print("Validation Set:")
print(classification_report(y_val, y_val_pred))
print("Accuracy:", accuracy_score(y_val, y_val_pred))

# Compute ROC AUC score for each class on validation set
roc_auc_scores_val = roc_auc_score(y_val, logistic_model.predict_proba(X_val), average='macro', multi_class='ovr')
print("ROC AUC Scores (Validation):", roc_auc_scores_val)

# Evaluation on testing set
print("\nTesting Set:")
print(classification_report(y_test, y_test_pred))
print("Accuracy:", accuracy_score(y_test, y_test_pred))

# Compute ROC AUC score for each class on testing set
roc_auc_scores_test = roc_auc_score(y_test, logistic_model.predict_proba(X_test), average='macro', multi_class='ovr')
print("ROC AUC Scores (Testing):", roc_auc_scores_test)

Training Set:
              precision    recall  f1-score   support

      Friday       0.00      0.00      0.00        34
      Monday       0.00      0.00      0.00        42
    No Visit       0.80      1.00      0.89       946
    Saturday       0.00      0.00      0.00        38
      Sunday       0.00      0.00      0.00        34
    Thursday       0.00      0.00      0.00        27
     Tuesday       0.00      0.00      0.00        34
   Wednesday       0.00      0.00      0.00        33

    accuracy                           0.80      1188
   macro avg       0.10      0.12      0.11      1188
weighted avg       0.63      0.80      0.71      1188

Accuracy: 0.7962962962962963
ROC AUC Scores (Training): 0.6882978537157325
Validation Set:
              precision    recall  f1-score   support

      Friday       0.00      0.00      0.00         3
      Monday       0.00      0.00      0.00         5
    No Visit       0.77      0.85      0.81        75
    Saturday       0.00    

### **Saving Model**

In [ ]:
with open('best_model.pkl', 'wb') as f:
  pickle.dump(logistic_model, f)
  print('Model Saved.')

Model Saved.


### **Get Inferences**

In [ ]:
# get sample
sample_idx = 88

# Get the visitor_id
train_visitor_id = df.loc[X_test.index, 'visitor_id']

# Create a new dataframe
visitor_pred_df = pd.DataFrame({'visitor_id': train_visitor_id, 'predicted_label': y_test_pred})

# Merge with X_test based on the index
result_df = pd.concat([X_test.reset_index(drop=True), visitor_pred_df.reset_index(drop=True)], axis=1)

# Display the result
result_df.loc[sample_idx, ['visitor_id', 'predicted_label']]

visitor_id             89
predicted_label    Monday
Name: 88, dtype: object

In [ ]:
# get sample
sample_idx = 7

# Display the result
result_df.loc[sample_idx, ['visitor_id', 'predicted_label']]

visitor_id               8
predicted_label    Tuesday
Name: 7, dtype: object

In [ ]:
# get sample
sample_idx = 39

# Display the result
result_df.loc[sample_idx, ['visitor_id', 'predicted_label']]

visitor_id              40
predicted_label    Tuesday
Name: 39, dtype: object

In [ ]:
# get sample
sample_idx = 67

# Display the result
result_df.loc[sample_idx, ['visitor_id', 'predicted_label']]

visitor_id               68
predicted_label    No Visit
Name: 67, dtype: object